# Tomoro assessment

## Intro

I outline how I am planning to tackle this assessment. At a high-level, I want to educate myself with what the paper is trying to achieve. After I have a basic understanding of what the authors are doing, I want to familiarise myself with the data. With this, basic modelling can act as our benchmark - which we can then enhance to reach an improved model.

Attack plan:
1. Read paper
2. Investigate what data we are working with
3. Think about what modelling is relevant
4. Start writing utils
    - in parallel look at paper + deep-dive data examples
6. Basic benchmarking
7. Improvements
8. Validation

## Preliminary paper overview

First pass of reading the paper. I present my notes below, for reference.

Authors create a new dataset, `ConvFINQA`
- Conversational  question/answer type dataset
- Questions require multi-step reasoning, specifically within the context of maths/finance
- Specialised domain versus general domain reasoning
- Answering questions requires context which is either given as text or tables

Multihop reasoning is a difficult task to model
- These questions require multiple operations until you reach the answer
    - Example: what is the % increase of variable X w.r.t to its value last year?
        - you need to know the variable X given at the current and last year; then calculate a % change
- Two types of questions
    - Simple: single multi-hop question (single isolated question that requires multiple operations to answer)
    - Hybrid: multiple multi-hop questions (require cross-question reasoning/ dependencies)
- Different math operations
    - Add, subtract, divide, multiply, power

How modelling is approached
- Neural symbolic approach: 
    - Combines a retriever to find the relevant facts/context; then a generator that uses question + context to decode the "reasoning program"
    - Reasoning program: They define it as a collection of operations. `op(arg1, arg2)` 
        - think of this as a functional expression of the reasoning required to solve the 
- Generative GPT like:
    - Relies on the context given during prompting "gold supporting facts"
        - Bad context, horrible output
    - Instructs the output to be like the reasoning program via examples
    - Investigates chain-of-thought

Key learnings:
- The model struggles with long reasoning chains.
- The model excels at number selection questions.
- The model suffers from the lack of domain knowledge.
- GPT-3 can do simple calculations by itself.
- GPT-3 performs better for its familiar program format.
- GPT-3 struggles with new complex task paradigms

## Preliminary data overview

In this section, I describe what I've learned through a preliminary data exploration. 


*Note: Data exploration can be messy. For this reason, I separate the actual work into a different script. Here, I report what I've learned. For further detail, please look at `scripts/data_exploration.ipynb`*

In [1]:
import pandas as pd

raw_data = pd.read_json('../data/train.json')
raw_data.iloc[0]

pre_text      [26 | 2009 annual report in fiscal 2008 , reve...
post_text     [year ended june 30 , cash provided by operati...
filename                                  JKHY/2009/page_28.pdf
table_ori     [[, Year ended June 30, 2009], [2008, 2007], [...
table         [[2008, year ended june 30 2009 2008, year end...
qa            {'question': 'what was the percentage change i...
id                               Single_JKHY/2009/page_28.pdf-3
annotation    {'amt_table': '<table class='wikitable'><tr><t...
qa_0                                                        NaN
qa_1                                                        NaN
Name: 0, dtype: object

Must knows:
1. Questions are stored under `.qa` for simple questions (single multi-hop) or `.qa_0` and `.qa_1` for hybrid questions (cross-question dependencies)
2. Each entry has an associated `.table` ; `.pre_text` and `.post_text` attribute that is used to construct our "context" - relevant during prompting
    - Basically this would look like a concatenation of `pre_text` + `table` + `post_text`
3. Each entry has also an `.annotation` attribute which contains the major information for the conversations.
    - Different information is captured if the entry is a simple question versus a hybrid question
    - Importantly, contains the `.dialogue_break` attribute, which splits the question(-s) into simpler subquestions, used to reach the answer
    - Contains `.qa_split` to tell you which info corresponds to which question in the case of hybrid questions
        - for example, exe_ans_list = [0,0,0,1,1] can be read as the first three elements of the lists correspond to `.qa_0` and the latter two correspond to `.qa_1`

In [43]:
print('Short example:\n')
print(f'    .qa.question:\n     {raw_data.iloc[0].qa.get("question")}\n')
print(f'    .annotation.dialogue_break:\n     {raw_data.iloc[0].annotation.get("dialogue_break")}\n')
print(f'    table:\n     {raw_data.iloc[0].table}\n')
print(f'    pre_text (excerpt):\n     {raw_data.iloc[0].pre_text[0][:100]}...\n')
print(f'    post_text (excerpt):\n     {raw_data.iloc[0].post_text[0][:100]}...\n')

Short example:

    .qa.question:
     what was the percentage change in the net cash from operating activities from 2008 to 2009

    .annotation.dialogue_break:
     ['what is the net cash from operating activities in 2009?', 'what about in 2008?', 'what is the difference?', 'what percentage change does this represent?']

    table:
     [['2008', 'year ended june 30 2009 2008', 'year ended june 30 2009 2008', 'year ended june 30 2009'], ['net income', '$ 103102', '$ 104222', '$ 104681'], ['non-cash expenses', '74397', '70420', '56348'], ['change in receivables', '21214', '-2913 ( 2913 )', '-28853 ( 28853 )'], ['change in deferred revenue', '21943', '5100', '24576'], ['change in other assets and liabilities', '-14068 ( 14068 )', '4172', '17495'], ['net cash from operating activities', '$ 206588', '$ 181001', '$ 174247']]

    pre_text (excerpt):
     26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business ...

    post_text (excerpt):
     

Important to note:
1. I've already found some bad examples - The data isn't perfect!
    - Cases where the maths operations don't seem to be correct
    - Some column names are repeated; effectively not being able to distinguish what numbers are
    - **This means that the accuracy metrics at the end should be taken with a grain of salt!**

2. We need to convert the table into something parsable by the LLM
    - Sometimes column names are supplied sometimes are not
    - I've spent sometime to think about it but there doesn't appear to be a global solution for this
    - You either assume that first entries are always column names or don't
    - Other option is to check the similarity of the first entries versus the rest of the table (in an attempt to understand if its a column name versus a cell value -- but this seems like a very computationally expensive operation)
3. Seems like the `pre_text` and `post_text` can be very noisy / dirty
    - Not always needed, but you can't know this apriori
    - There's the option of the `.gold_inds` but this feels like cheating as its the perfect retrieval

## Utils and functions

The relevant utilities can be found in the `utils` folder, which at a high level include the data preprocessing infra, model infra, and other evaluator functions.

Please look at each respective file for explanation of their contents - this report will not go through in detail, unless relevant/ interesting.

## Basic benchmarking

I want to write the basic functionality to parse all this information coherently, and run a few benchmarks.
I'm thinking I'll take the API approach as my computer cannot run much...

I am relying to achieve most of the performance improvement through prompt engineering. I present here the thought process of how each prompt was structured and the underlying strategy.

I am using Deepseek's models as they are the cheapest at the moment of writing. I have allocated 5-10$ for this assessment and generate an API key to use their models. I do this by using OpenAI's SDK as suggested by the original Deepseek docs.

### Prompt engineering

There are some basics that we want to force our LLM to do. 
1. Use only the context it was provided and not rely on any external information
2. We want to enforce some output format to be able to easily extract the answer

#### Prompt 01

The first prompt, naively asks the LLM to give the numerical output. This was my first attempt to just get a basic understanding if the `utils` work in general and what the LLM can or cannot do.

I used a system prompt, asking the LLM to act as a financial analysis expert - this is because the questions revolve around finance, and if not directly relevant, still use mathematics. All of which are mostly found in financial data. This is our attempt to trigger the relevant weights that correspond to that type of "thinking".

*It is important to note that asking the LLM to do maths directly isn't smart. This is apparent immediately, and it will be resolved but we are still in exploration mode. Even if its reasoning it correct, the actual evaluation of operations is unlikely to be correct. Purely out of how LLMs work underneath + the data it has seen.*

```
SYSTEM_PROMPT_V1 = r"""
    Act as a financial analysis specialist. Your responses must:
    1. Strictly use only the contextual information provided by the user
    2. Deliver the final answer in this exact format:
        - Unitless numerical value
        - Enclosed in \boxed{} LaTeX formatting

    Never reference external knowledge or assumptions. 
    Convert all scaled values to absolute numbers during calculations,
    but omit units in the final answer.
    """
```


#### Prompt 02

Very similar to the first, still do not ask it to give maths operators - still evaluates the answer directly. The difference here is I want to extract its reasoning. This was an attempt to see what the output is like, and consider if I can use it to extract it for the user. The user likely wants to know where in the document that information is present.

This evolves a bit more, in the later stages so won't be described in detail here. Look for how context is extracted in the `Structured output` section.

This allowed me to explore a bit how the thought process is presented/ structured.

```
SYSTEM_PROMPT_V2 = r"""
    Act as a financial analysis specialist. Your responses must:
    1. Strictly use only the contextual information provided by the user
    2. Explicitly show your logical reasoning process through sequential step-by-step explanations
    3. Deliver the final answer in this exact format:
        - Unitless numerical value
        - Enclosed in \boxed{} LaTeX formatting

    Never reference external knowledge or assumptions. 
    Convert all scaled values to absolute numbers during calculations, 
    but omit units in the final answer.
    """
```


#### Prompt 03

We now ask the LLM to use a selection of pre-defined maths operations, and NEVER evaluate the answer. This means we expect the LLM to give an output of looking like `add(a, b)` or similar.

For this to work, we slightly modify the prompt but keep all previous asks (use context only + ignore units + output format).

I've researched a bit on how to structure the language and landed on this approach. Language here can be varied and is up to the developer's approach. It does impact the LLM but it is difficult to know exactly what words/language will lead to the best output without iteratively testing each one.

Importantly:
1. I define the operators
2. I explicitly mention that I am expecting nested operators rather than new lines/ evaluated intermediate steps
3. I define that each operator ONLY takes two arguments 
4. Give a short example

```
SYSTEM_PROMPT_V3 = r"""
    Act as a financial computation engine. Required behavior:
    1. Input Processing:
    - Use ONLY context provided in the query
    - Never incorporate external data or assumptions
    2. Calculation Methodology:
    - Perform and display calculations by using ONLY these Python-style operators:
        - add(a, b) → a + b
        - subtract(a, b) → a - b
        - multiply(a, b) → a * b
        - divide(a, b) → a / b
        - power(a, b) → a^b
    - Each operator must have EXACTLY two arguments
    3. Output Requirements:
    - Final answer must be:
        - A nested combination of allowed operators
        - In unevaluated functional form
        - Expressed as \boxed{operator(...)} LaTeX
    - Include intermediate unit normalization calculations

    Example: For "Revenue per share - Cost per share = (5,000,000 revenue / 2,000,000 shares) - $5"
    Acceptable: \boxed{subtract(divide(5000000, 2000000), 5)}
    Unacceptable: \boxed{2.5 - 5} or \boxed{-2.5}
    """
```

With this, I write a function called `evaluate_maths` that will be taking the string output of the LLM, and using python's `eval` method to evaluate these operators. I also write a dictionary of these maths operators (exactly as seen in the prompt) for python to use.


#### Prompt 04: Structured output

**Note: Structured output is usually coded by providing a json schema + pydantic objects. However, based on Deepseek's documentation, they expect this to be given during prompting.**

**See here: https://api-docs.deepseek.com/guides/json_mode**

**For this reason, I provide a prompt that handles structured outputs and will consider if a pydantic implementation is required (if say I use other model providers)**

This was necessitated after a lot of investigation of how the outputs are produced. 

Structured outputs is the mechanism of forcing your LLM to produce its output in a very defined way. The LLM is given a `json` schema and it's output will abide to it. With this, we can force specific attributes, for example quoting where the relevant text is, the thought process, and the final `program` to be evaluated. Without this approach, the LLM sometimes hallucinates. This is also a cause of "randomness" in a way (even if you set temperature to 0.0)

```
SYSTEM_PROMPT_V4 = r"""
    Act as a financial computation engine that outputs valid JSON. Required behavior:
    1. Input Processing:
    - Use ONLY context provided in the query
    - Never incorporate external data or assumptions
    2. Calculation Methodology:
    - Perform and display calculations by using ONLY these Python-style operators:
        - add(a, b) → a + b
        - subtract(a, b) → a - b
        - multiply(a, b) → a * b
        - divide(a, b) → a / b
        - power(a, b) → a^b
    - Each operator must have EXACTLY two arguments
    3. JSON Output Requirements:
    - Structure response as valid JSON with this schema:
        {
            "user_question": "string",
            "user_context": "string",
            "reasoning": ["step1", "step2", ..., "stepN"],
            "final_answer": "boxed_expression"
        }
    - Maintain atomic values in JSON (no complex objects)
    - Escape special characters properly
    - final_answer must use: \boxed{operator(...)} format
    
    4. Compliance:
    - Strictly follow JSON syntax
    - No markdown formatting
    - No additional explanations outside JSON structure

    Example valid response:
    {
        "user_question": "Calculate profit per share given 5M revenue and 2M shares with $5 fixed cost",
        "user_context": "[Row 1] Revenue: 5,000,000\n[Row 2] Shares: 2,000,000\n[Row 3] Fixed cost per share: 5",
        "reasoning": [
            "1. Revenue per share - Cost per share", 
            "2. Convert 5M revenue to 5,000,000",
            "3. Divide revenue by shares: 5,000,000/2,000,000",
            "4. Subtract fixed cost per share from revenue per share",
            "4. Use subtract() for subtraction and divide() for division"
        ],
        "final_answer": "\boxed{subtract(divide(5000000, 2000000), 5)}"
    }
    """
```